In [1]:
# 라이브러리 선언

from selenium import webdriver

from selenium.webdriver.common.keys import Keys

import bs4

import pandas as pd

import time



# 드라이버 위치 설정

driver_loc = "./chromedriver.exe"

 

# 드라이버 옵션 설정

options = webdriver.ChromeOptions()


options.add_argument("window-size=2280x1440") # 파이썬이 크롬창을 띄울 것인데 창 크기 선택

 

# 웹 드라이버 정의

driver = webdriver.Chrome(driver_loc, options=options)

 

# 웹페이지 이동

# URL 정의

smartwatchUrl = "http://www.danawa.com/"

driver.get(smartwatchUrl)

 

# 스마트워치 사이트로 이동

categoryXpath = '//*[@id="danawa_header"]/div[2]/div/div[1]/a/span[2]'

categoryMenu = driver.find_element_by_xpath(categoryXpath)

categoryMenu.click()

time.sleep(0.5)

 

mobileXpath = '//*[@id="sectionExplodeLayer"]/div[1]/ul/li[3]/a'

mobileMenu = driver.find_element_by_xpath(mobileXpath)

webdriver.ActionChains(driver).move_to_element(mobileMenu).perform()

time.sleep(0.5)

 

smartBandXpath = '//*[@id="categoryExplodeLayer12"]/div[2]/div[2]/ul/li[3]/a'

smartBandBtn = driver.find_element_by_xpath(smartBandXpath)

smartBandBtn.send_keys(Keys.ENTER)


rowList = []

titleList = []

titleList.append('제품명')

titleList.append('최저가')

titleList.append('사이트주소')

 

for i in range(0, 999):

    try:

        # 웹페이지 이동 설정

        pageSelector = '#productListArea > div.prod_num_nav > div > div > a:nth-child(' + str(i + 1) + ')'

        page = driver.find_element_by_css_selector(pageSelector)

        page.click()

        time.sleep(0.5)

    except:

        break

        

    # 웹페이지 소스코드 변수에 저장

    smartBandHtml = driver.page_source

    

    # 태그 정보만 추출하여 변수에 저장

    bs = bs4.BeautifulSoup(smartBandHtml, 'html.parser')

    divTag1 = bs.find(name = 'div', attrs = {"class":"main_prodlist main_prodlist_list"})

    divTags2 = divTag1.findAll(name = "div", attrs = {"class":"prod_main_info"})

    

    itemNum = len(divTags2)

    

    columnList = []

    

    for j in range(0, itemNum):

        columnList.append(divTags2[j].find(name = "div", attrs = {"class":"prod_info"}).find(name = 'p', attrs = {"class":"prod_name"}).find(name = 'a').text.strip())

        

        driver.find_elements_by_name("productName")[j].click()

        

        time.sleep(0.5)

        

        driver.switch_to.window(driver.window_handles[(1)])

        

        time.sleep(0.5)

        # 웹페이지 소스코드 변수에 저장

        smartBandHtml2 = driver.page_source

 

        # 태그 정보만 추출하여 변수에 저장

        bs = bs4.BeautifulSoup(smartBandHtml2, 'html.parser')

        

        if bs.find('tr', {'class':'lowest'}) != None:

            # 최저가 정보 추출

            spanTag = bs.find('span', {'class':'lwst_prc'})

            columnList.append(spanTag.find('em', {'class':'prc_c'}).text.replace(',',""))

 

            # 최저가 구매사이트 추출

            siteTag = bs.find('tr', {'class':'lowest'})

            columnList.append(siteTag.find('a')['href'])

        else:

            nopriceTag = bs.find('div',{'class':'lowest_area lowest_blank'})

            columnList.append(nopriceTag.find('strong', {'class':'pnt'}).text)

            columnList.append('')

        

        # 테이블 정보를 포함한 상위태그 추출

        tableTag = bs.find('table', {'class':'spec_tbl'})

        tbodyTag = tableTag.find('tbody')

 

        tdTags = tbodyTag.findAll('td')

        thTags = tbodyTag.findAll('th', {'class':'tit'})

 

        for k in range(0, len(tdTags)):

            if thTags[k].text != '':

                if j == 0 and i == 0:

                    titleList.append(thTags[k].text)

                    

                if k == 0:

                    columnList.append(tdTags[k].text.strip().split(' ')[0])

                else:    

                    columnList.append(tdTags[k].text)

 

        rowList.append(columnList)

        

        columnList = []

        

        driver.close()

        

        driver.switch_to.window(driver.window_handles[0])


# 데이터 가공

## 중복된 데이터 제거를 위해 길이가 82인 row의 7번째 인덱스 삭제

delList = []

rowListLen = len(rowList)

for i in range(0, rowListLen):

    if len(rowList[i]) == 82:

        delList.append(i)

delColNum = 7

for i in delList:

    del rowList[i][delColNum]


## 연주차컬럼 정제. 20XX년 09월 -> 20XX09

yearweekIndex = 4

for i in range(0, len(rowList)):

    rowList[i][yearweekIndex] = rowList[i][yearweekIndex].strip().replace('년 ','').replace('월','')
# 데이터 저장

smartBand = pd.DataFrame(rowList, columns = titleList)

smartBand.to_csv('smartBand.csv', index=False)

 



In [2]:
# 데이터 저장

smartBand = pd.DataFrame(rowList, columns = titleList)

smartBand.to_csv('smartBand.csv', index=False)